In [3]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from modules import solver
from pyballistics import ozvb_lagrange, get_db_powder, get_powder_names
from tqdm.notebook import tqdm, trange

In [2]:
print(solver.__doc__)

 version 1.0

This module solves inverse problem of internal ballistics




## Исходные данные
задаются начальные данные по орудию и формируется словарь

__Основные парметры__

In [3]:
d = 125*1e-3          # калибр м
q = 4.85           # вес снаряда кг
velocity_pm = 1700         # дульная скорость снаряда
n_s = 1           # нарезное орудие
max_pressure = 1000*1e6    # максимальное давление Па
tube_lenght = 5543*10e-3      # длина трубы  

__Дополнительные параметры__

In [4]:
# !! fix it
p_fors = 10*1e6      #давление форсирования (гладкоствольные 7-15 МПа)

#нормализация (в данный момент не используется)
norm_ro = 1000 
norm_w = 1

__Словарь__

In [5]:
initial_dict_static =  {
   'powders': 
    [
       {'omega': None, 'dbname': None}
    ],
  'init_conditions': 
    {
       'q': q,
       'd': d,
       'W_0': None ,
       'phi_1': 1.0,
       'p_0': p_fors, 
       'n_S': 1
    },
  'igniter': 
    {
       'p_ign_0': 5000000.0 #check it 
    },
  
  'meta_lagrange': 
    {
       'CFL': 0.9, 
       'n_cells': 300
    },
  'stop_conditions': 
    {
       'x_p': tube_lenght,
       'steps_max': 8000,   
       't_max': 0.05,
       'p_max': max_pressure,
        'v_p': velocity_pm,
    }
 }

## Решение обратной задачи

__Инициализация массивов под решение__

In [6]:
powders_names = get_powder_names()
n = len(powders_names)
wq_0 = np.random.rand(n)*0.8 + 0.1
ro_0 = np.random.rand(n)*800 + 100

In [7]:
optim_parametrs = list()

__Цикл решения__:

    1. Внешний цикл по порохам
    2. решение церез штафную функцию

In [8]:
%%time

for i in trange(2):
    initial_dict = copy.deepcopy(initial_dict_static)
    initial_dict['powders'][0]['name'] = powders_names[i]
    initial_dict['powders'][0]['omega'] = wq_0[i]*q
    initial_dict['init_conditions']['W_0'] = wq_0[i]*q/ro_0[i]
    optim_parametrs.append(solver.solution(initial_dict))


  0%|          | 0/2 [00:00<?, ?it/s]

error


KeyError: 'layers'

In [9]:
5 * None

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

__Выбор лучшего решения__

In [ ]:
hi = 2
d = 0.125

In [ ]:
l_2 = 0.55*d if np.sqrt(hi) <= 1.25 else 2*d

In [ ]:
del_radius = np.array([0, 12, 2, 15])
d = 20

In [ ]:
for i in range(len(del_radius)):
    del_radius[i] = max(del_radius[i], 0.5*d) 

In [ ]:
del_radius

In [ ]:
a = [5, 8,1, 89]
b = np.array([5, 8, 6])

In [ ]:
a +b

In [ ]:
x = np.array([4, 6, 8, 9, 10])

In [ ]:
x[1:] - x[:-1]

In [7]:
srt1 = 'p'
str2 = 'a'
arr = ['x', 'p', 'd']

In [9]:
srt1 in arr 

True

In [35]:
a = np.array([
    [3, 6, 7, 4, 8],
    [5, 10, 3, 7, 9],
    [17, 9, 1, 16, 7]
])

In [36]:
index = np.argmax(a)
index

10

In [37]:
a.shape

(3, 5)

In [38]:
border = index%a.shape[1]
count = index//a.shape[1]

In [39]:
border, count

(0, 2)

In [40]:
a[count, border]

17

In [42]:
np.argmax(a[1:], axis=0)

array([1, 0, 0, 1, 0], dtype=int64)